In [3]:
import os
os.environ['DISPLAY'] = ':99'


In [4]:
from open3d.io import read_triangle_mesh
from open3d.visualization import Visualizer, MeshColorOption

geometry = read_triangle_mesh("/data/wlsgur4011/part_assembly/example_data/artifact/39087_sf/fractured_0/piece_0.obj")
geometry.compute_vertex_normals()
print(geometry)

vis = Visualizer()
import jhutil; jhutil.jhprint(1111, vis.get_render_option())
vis.create_window()
vis.add_geometry(geometry)
vis.get_render_option().mesh_color_option = MeshColorOption.Normal
vis.run()
vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
TriangleMesh with 6641 points and 15248 triangles.
1111  "None"
[Open3D WARNING] GLFW Error: GLX: No GLXFBConfigs returned
[Open3D WARNING] GLFW Error: GLX: Failed to find a suitable GLXFBConfig
[Open3D WARNING] Failed to create window


AttributeError: 'NoneType' object has no attribute 'mesh_color_option'

In [1]:
from open3d import *

geometry = read_triangle_mesh("TestData/knot.ply")
geometry.compute_vertex_normals()
print(geometry)

vis = Visualizer()
vis.create_window()
vis.add_geometry(geometry)
vis.get_render_option().mesh_color_option = MeshColorOption.Normal
vis.run()
vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


: 

: 

In [1]:
import numpy as np
import trimesh
import pyrender

def generate_normal_map(mesh_file, camera_position=(1, -1, 1)):
    # Load the 3D mesh
    mesh = trimesh.load(mesh_file)

    # Create a Pyrender mesh from the Trimesh object
    pr_mesh = pyrender.Mesh.from_trimesh(mesh)

    # Set up the scene with the mesh and a camera
    scene = pyrender.Scene()
    scene.add(pr_mesh)

    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    camera_pose = np.eye(4)
    camera_pose[:3, 3] = camera_position
    scene.add(camera, pose=camera_pose)

    # Set up the renderer with an off-screen buffer
    renderer = pyrender.OffscreenRenderer(512, 512)
    
    # Render the scene to get the depth buffer and the color buffer
    color_buffer, depth_buffer = renderer.render(scene)
    
    # Use numpy gradient function to compute the normal map
    normals = np.gradient(depth_buffer)
    
    # Normalize the normal map
    normal_map = normals / np.linalg.norm(normals, axis=0)

    # Convert to 8-bit color and return the normal map
    return (normal_map * 255).astype(np.uint8)

# Usage
normal_map = generate_normal_map('/data/wlsgur4011/part_assembly/example_data/artifact/39087_sf/fractured_0/piece_0.obj')


NoSuchDisplayException: Cannot connect to "None"

In [12]:
import trimesh
import matplotlib.pyplot as plt
import numpy as np

def create_normal_map(mesh_file):
    # Load the mesh
    mesh = trimesh.load_mesh(mesh_file)

    # Compute the vertex normals
    normals = mesh.vertex_normals
    import jhutil; jhutil.jhprint(1111, normals)

    # Map the normals from [-1, 1] to [0, 255]
    normals = ((normals + 1) / 2 * 255).astype(np.uint8)
    import jhutil; jhutil.jhprint(2222, normals)

    # Reshape the normals into an image
    import jhutil; jhutil.jhprint(3333, mesh.vertices.shape[0], mesh.vertices.shape[1], 3)
    normal_map = normals.reshape(mesh.vertices.shape[0], mesh.vertices.shape[1], 3)

    return normal_map

def save_image(normal_map, output_file):
    plt.imshow(normal_map)
    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)

normal_map = create_normal_map('/data/wlsgur4011/part_assembly/example_data/artifact/39087_sf/fractured_0/piece_0.obj')
save_image(normal_map, 'normal_map.png')


1111  "array[6641, 3] n=19923 (0.2Mb) x∈[-1.000, 1.000] μ=0.029 σ=0.576"
2222  "array[6641, 3] u8 n=19923 (19Kb) x∈[0, 255] μ=130.759 σ=73.485"
3333  "6641" "3" "3"


ValueError: cannot reshape array of size 19923 into shape (6641,3,3)

In [7]:
import pyglet
from pyglet.gl import *
from pyrr import Matrix44, Vector3

# Load the mesh
mesh = pyglet.resource.model('model.obj')

# Set up an offscreen buffer for rendering
buffer = pyglet.image.get_buffer_manager().get_color_buffer()

# Set up OpenGL
glEnable(GL_DEPTH_TEST)

glClearColor(0.5, 0.5, 1, 0)  # clear to normal-neutral color

# Set up a perspective projection
glMatrixMode(GL_PROJECTION)
glLoadIdentity()
gluPerspective(60, 1, 0.1, 1000)
glMatrixMode(GL_MODELVIEW)

# Draw the mesh from multiple directions
for angle in range(0, 360, 90):
    glLoadIdentity()
    
    # Position the camera
    glTranslatef(0, 0, -3)
    glRotatef(angle, 0, 1, 0)

    # Clear the buffer
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Draw the mesh
    mesh.draw()

    # Read the pixels from the buffer and write them to an image
    data = buffer.get_image_data().get_data('RGB', buffer.width * 3)
    img = pyglet.image.ImageData(buffer.width, buffer.height, 'RGB', data)
    img.save('normalmap_%s.png' % angle)


NoSuchDisplayException: Cannot connect to "None"